# Neural Style Transfer

> Neural style transfer is an optimization technique where two images—a *content* image and a *style reference* image (such as an artwork by a famous painter)—are blend together to create an output image which would look like the content image, but “painted” in the style of the style reference image.



In [0]:
# Importing libraries
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# Utility Functions

In [0]:
def show_img(img, title):
  if len(img.shape) > 3:
    img = tf.squeeze(img, axis=0)
  
  plt.title(title)
  plt.imshow(img)

In [0]:
def show_tensor_as_img(tensor):
  img = tensor.numpy()
  show_img(img, "Generated Image")

In [0]:
def save_img(img, img_path):
  if len(img.shape) > 3:
    img = tf.squeeze(img, axis=0)
  img *= 255.0 
  tf.keras.preprocessing.image.save_img(img_path, img)

In [0]:
def clip_img(img, min_val=0.0, max_val=1.0):
  return tf.clip_by_value(img, min_val, max_val, name="clipping_img")

In [0]:
def scale_img(img, max_size=512):
  shape = tf.cast(img.shape[:-1], dtype=tf.float32)
  max_dim = max(shape)
  scale = tf.constant(max_size / max_dim, dtype=tf.float32)
  new_shape = tf.cast(shape * scale, dtype=tf.int32)
  resized_img = tf.image.resize(img, new_shape, name="resizing_img")
  return resized_img[tf.newaxis, :]

In [0]:
def prepare_img(img_loc, img_name="input_img"):
  raw_img = tf.io.read_file(img_loc)
  img = tf.io.decode_image(raw_img, channels=3, dtype=tf.float32, 
                           name= img_name)
  scaled_img = scale_img(img)
  return scaled_img

# Model Definition & Losses

In [0]:
def gram_matrix(feature_map):
  b, h, w, c = feature_map.shape
  reshaped_map = tf.reshape(feature_map, [w * h, c])
  normalization_factor = tf.cast(h * w * c, dtype=tf.float32)
  gram_matrix = tf.linalg.einsum('bijc,bijd->bcd', feature_map, feature_map, 
                                 name="gram_matrix")
  return gram_matrix / normalization_factor

In [0]:
def content_style_loss(generated_outputs, content_targets, style_targets, 
                       content_weight, style_weight):
  generated_content_outputs = generated_outputs["content_map"]
  generated_style_outputs = generated_outputs["style_map"]
  content_loss = [tf.reduce_mean((content_targets[layer_name] - generated_content_outputs[layer_name])**2)
                                 for layer_name in generated_content_outputs.keys()]
  content_loss = tf.reduce_mean(content_loss, name="content_loss")
  
  style_loss = [tf.reduce_mean((style_targets[layer_name] - generated_style_outputs[layer_name])**2)
                                 for layer_name in generated_style_outputs.keys()]
  style_loss = tf.reduce_mean(style_loss, name="style_loss")

  total_loss = content_weight * content_loss + style_weight * style_loss
  return total_loss

In [0]:
def train_step(img, model, optimizer, content_targets, style_targets, 
               content_weight, style_weight):
  with tf.GradientTape() as tape:
    generated_outputs = model(img)
    loss = content_style_loss(generated_outputs, content_targets, style_targets,
                              content_weight, style_weight)

  gradients = tape.gradient(loss, img)
  optimizer.apply_gradients([(gradients, img)])
  img.assign(clip_img(img))
  return loss

In [0]:
def get_custom_vgg_model(layer_names):
  # Loading pre-trained VGG19 network
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  output_layers = [vgg.get_layer(name).output for name in layer_names]
  model = tf.keras.Model(vgg.input, output_layers)
  return model

In [0]:
class NSTModel(tf.keras.Model):

  def __init__(self, content_layers, style_layers):
    super(NSTModel, self).__init__()
    self.custom_vgg = get_custom_vgg_model(content_layers + style_layers)
    self.preprocessor = tf.keras.applications.vgg19.preprocess_input
    self.content_layers = content_layers
    self.style_layers = style_layers
    self.n_content_layers = len(content_layers)

  def call(self, inputs):
    inputs = inputs * 255.0
    preprocessed_inputs = self.preprocessor(inputs)
    outputs = self.custom_vgg(preprocessed_inputs)
    content_outputs, style_outputs = outputs[:self.n_content_layers], \
                                      outputs[self.n_content_layers:]
    style_corr_values = [gram_matrix(feature_map) 
                        for feature_map in style_outputs]
    content_map = {layer_name: layer_output for layer_name, layer_output 
                   in zip(self.content_layers, content_outputs)}
    style_map = {layer_name: layer_output for layer_name, layer_output
                 in zip(self.style_layers, style_corr_values)}
    return {"content_map": content_map, "style_map": style_map}

# Content & Style Layers

In [0]:
# Choosing our content layers
content_layers = ["block5_conv2"]

# Choosing out style layers
style_layers = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", 
                "block5_conv1"]

# Style Transfer Begins

In [0]:
# Creating object for our model
model = NSTModel(content_layers, style_layers)

# Creating our model optimzer
adam_opt = tf.optimizers.Adam(learning_rate=0.001, beta_1=0.99, epsilon=1e-1)

In [0]:
# Loading our content image
content_img = prepare_img("/content/bhai.jpeg")

# Preparing our target values
content_targets = model(content_img)["content_map"]

# Preparing our output image
output_img = tf.Variable(content_img)

In [0]:
# Loading our style image
style_img = prepare_img("/content/style1.jpg")

# Preparing our target values
style_targets = model(style_img)["style_map"]

In [0]:
plt.subplot(1, 2, 1)
show_img(content_img, "Content Image")
plt.subplot(1, 2, 2)
show_img(style_img, "Style Image")

In [0]:
# Defining our hyper-parameters
content_weight = 1e-4
style_weight = 1e-3
n_iter = 300

# Running gradient descent on output image
for cur_iter in range(1, n_iter+1):
  loss = train_step(output_img, model, adam_opt, content_targets, style_targets,
                    content_weight, style_weight)
  if cur_iter % 50 == 0:
    print("Loss after {} iteration is: {:.4f}".format(cur_iter, loss))
    save_img(output_img.numpy(), "/content/gen_img_{}.jpeg".format(cur_iter))

show_tensor_as_img(output_img)